In [7]:
import os
import cv2
import numpy as np

import torch
from torch.utils.data import DataLoader

from data.ship_dataset import ImgDataset, collate_fn

In [ ]:
def custom_loss(y_pred, y_true):
    confi_loss = nn.BCELoss()
    
    if confi_loss == 1:
        loss = confi_loss
    else:
        bbox_loss = nn.MSELoss()
        class_loss = nn.BCEWithLogitsLoss()

        loss = bbox_loss(y_pred[:4], y_true[:4]) + confi_loss + class_loss(y_pred[5:], y_true[5:])

    return loss

In [ ]:
from torch import nn


class YoloV3Loss(nn.Module):
    ## __init__() will serve as the constructor of out loss function.
    def __init__(self):
        super().__init__()
        
        self.mse = nn.MSELoss()
        ## pos_weight=?
        self.objBCE = nn.BCEWithLogitsLoss()
        self.multiBCE = nn.BCEWithLogitsLoss()


    ## forward() is where we will perfrom the calculations of the loss function.
    ## ## we can also define backward() function, which take control of how the gradient is calculated,
    ## ## but it is recommended to let ""PyTorch's autograd""" calculate the gradients automatically.
    def forward(self, pred, target, anchors):
        dtype = pred.dtype
        device = pred.device

        # pred = pred.reshape(-1, 85)

        ## 
        obj_loss = self.objBCE(pred[:, 4], target[:, 4])
        class_loss = torch.tensor(0., dtype=dtype, device=device)
        
        if self.do_ignore(pred, target):
            loss = obj_loss

            ## loss * ???
            return loss
        else:
            ## MSE Loss for bbox coordination
            bbox_x_loss = self.mse(pred[:, 0], target[:, 0])
            bbox_y_loss = self.mse(pred[:, 1], target[:, 1])
            bbox_w_loss = self.mse(pred[:, 2], target[:, 2])
            bbox_h_loss = self.mse(pred[:, 3], target[:, 3])
            bbox_loss = bbox_x_loss + bbox_y_loss + bbox_w_loss + bbox_h_loss

            ## multi-class BCE Loss for bbox class
            class_num = pred.shape[1] - 5
            for i in range(class_num):
                class_loss += self.multiBCE(pred[:, i], target[:, i])
            class_loss /= class_num     ## Can I use the mean...?
            
            ## Total Loss
            loss = bbox_loss + obj_loss + class_loss

            ## loss * ???
            return loss

    
    def do_ignore(self, pred, target):

        return False


In [ ]:
dataset_option = {  "DATASET": {
                        "NAME": "ship",
                        "ROOT": "../datasets/ship",
                        "CLASSES": {
                               "선박": 0, "부표": 1, "어망부표": 2,
                               "해상풍력": 3, "등대": 4, "기타부유물" : 5
                        }
                     }
                 }

batch_size = 32
epochs = 1

In [5]:
model = Net()
loss_function = YoloV3Loss()

In [ ]:
valid_dataset = ImgDataset(dataset_option, split="valid")
valid_loader = DataLoader(valid_dataset, batch_size, collate_fn=collate_fn)

In [ ]:
for epoch in range(epochs):
    for batch_img, batch_label, batch_img_path in valid_loader:
        output = model(batch_img)

        loss = ( loss_function(output[0], batch_label, ...)
               + loss_function(output[1], batch_label, ...)
               + loss_function(output[2], batch_label, ...) )

        loss.backward()